### 폐병 수술(Thoracic Surgery)
    - 케라스 참고: https://tykimos.github.io/2017/07/09/Training_Monitoring/

In [7]:
# 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving ThoraricSurgery.csv to ThoraricSurgery.csv


In [9]:
uploaded.keys()

dict_keys(['ThoraricSurgery.csv'])

In [10]:
filename = list(uploaded.keys())[0]

In [11]:
import numpy as np
import pandas as pd

In [13]:
df = pd.read_csv(filename, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0


- Train/Test dataset 분리

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.values[:, :-1], df.values[:, -1], stratify=df.values[:, -1], random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((352, 17), (118, 17), (352,), (118,))

- Random 값 정리

In [15]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

- 모델 정의
    - add(): 케라스 모델에 층을 추가하는 메서드
        - 케라스 모델의 add()메서드는 keras.layers 패키지 아래에 있는 층의 객체를 입력받아 신경망 모델에 추가한다
        - add() 메서드를 호출하여 전달한 순서대로 층이 차례대로 늘어난다.
    - summary(): 케라스 모델의 정보를 출력하는 메서드
        - 모델에 추가된 층의 종류와 순서, 모델 파라미터 개수를 출력
    - Sequential: 케라스에서 신경망을 만드는 클래스
        - 이 클래스의 객체를 생성할 때 신경망 모델에 추가할 층을 지정할 수 있다. 추가할 층이 1개 이상일 경우 파이썬 리스트로 전달한다.
    - Dense: 신경망에서 가장 기본 층인 밀집층을 만드는 클래스
        - activation: 사용할 활성화 함수를 지정
            - sigmoid, softmax
    - 케라스의 Sequential 클래스에 맨 처음 추가되는 층에는 input_shape 매개변수로 입력의 크기를 지정해야 한다.
    - compile(): 모델 객체를 만든 후 훈련하기 전에 사용할 손실 함수와 측정 지표 등을 지정하는 메서드
        - loss: 손실함수 지정
            - 이진분류: binary_crossentropy
            - 다중분류: categorical_crossentropy
            - 클래스 레이블이 정수일 경우: sparse_categorical_croeeentropy
            - 회귀 모델일 경우: mean_square_error
        - metrics: 훈련과정에서 측정하고 싶은 지표를 지정할 수 있다.
            - 측정 지표가 1개 이상일 경우 리스트로 전달'
            - 케라스는 모델이 훈련할 때 기본적으로 에포크마다 손실값을 출력.
                - 손실이 줄어드는 것을 보고 훈련이 잘 되었다는 것을 알 수 있지만 정확도를 함께 출력하면 더 좋다!
                -   이를 위해, metrics 매개변수에 정확도 지표를 의미하는 'accuracy'를 지정

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
model = Sequential()                                   # input layer는 따로 주지 않음
model.add(Dense(30, input_dim=17, activation='relu'))  # 케라스에서 신경망의 첫 번째 층은 inpit_shape 매개변수로 입력의 크기를 반드시 지정해주어야 한다.
                                                       # activation == 활성화 함수 /// input에서 전달하는 매개체를 relu로 사용하겠다는 의미
                                                       # 은닉층(Hidden layer)
model.add(Dense(1, activation='sigmoid'))              # 출력층(Output layer) 
                                                       # 이진분류 => activation='sigmoid'
model.summary()                                        # 입력층(17개 == a) * 은닉층(30개 == x) = 510개 + 30개(절편 == b) = 540개 
                                                       # dense(Dense) ----------(None,30)----------540
                                                       #                         None: 샘플 개수가 아직 정해지지 않았기 때문에 None
                                                       #                            케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 한 번에 모두 사용하지 않고 잘게 나누어 여러번에 걸쳐 경사하강법 단계를 수행한다
                                                       #                            fit() 메서드에서 batch_size 매개 변수로 바꿀 수 있다.
                                                       #                            따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정
                                                       #                            배치 차원: 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원
                                                       #                         30: 은닉층의 뉴런 개수를 30개로 두어서, 30개의 출력이 나옴 
                                                       #                         540: 입력층(17개 == a) * 은닉층(30개 == x) = 510개 + 30개(절편 == b) = 540개
                                                       # dense_1(Dense) ----------(None,1)----------31
                                                       #                           1: 출력 뉴런 개수
                                                       #                           31: 30개의 은닉층 뉴런과 1개의 은닉층 뉴런

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                540       
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(
    optimizer='adam',                   # cf. 회귀: 경사하강법 (최소 지점을 찾아가는 방법)
    loss = 'binary_crossentropy',       # 손실함수: 이진분류 'binary_crossentropy'
    metrics = ['accuracy']             
)

- 모델 학습
    - fit(): 모델을 훈련하는 메서드
        - 첫 번째와 두 번째 매개변수에 입력과 타깃 데이터를 전달
        - epochs: 전체 데이터에 대해 반복할 에포크 횟수를 지정

In [20]:
model.fit(X_train, y_train ,batch_size=30, epochs=30, validation_split=0.2)        # batch_size => 데이터를 한꺼번에 몇 개를 받을 것인지 결정
                                                                                   # 배치 사이즈가 크면 속도는 빠르지만, 메모리 소모가 그만큼 빠르다.
                                                                                   # validation_split => 학습(weight값 갱신) -> 검증(20%)
                                                                                   # 검증 세트의 점수는 훈련 세트 점수보다 조금 낮은 것이 일반적이다. 

Epoch 1/30
10/10 [==============================] - 1s 24ms/step - loss: 42.6976 - accuracy: 0.1957 - val_loss: 43.9042 - val_accuracy: 0.2817
Epoch 2/30
10/10 [==============================] - 0s 5ms/step - loss: 34.1635 - accuracy: 0.2740 - val_loss: 35.1561 - val_accuracy: 0.3521
Epoch 3/30
10/10 [==============================] - 0s 5ms/step - loss: 26.6377 - accuracy: 0.3737 - val_loss: 27.0327 - val_accuracy: 0.2958
Epoch 4/30
10/10 [==============================] - 0s 4ms/step - loss: 19.1766 - accuracy: 0.3950 - val_loss: 19.6114 - val_accuracy: 0.3662
Epoch 5/30
10/10 [==============================] - 0s 5ms/step - loss: 12.9497 - accuracy: 0.4377 - val_loss: 12.6753 - val_accuracy: 0.3944
Epoch 6/30
10/10 [==============================] - 0s 7ms/step - loss: 7.6402 - accuracy: 0.5125 - val_loss: 7.0563 - val_accuracy: 0.4648
Epoch 7/30
10/10 [==============================] - 0s 5ms/step - loss: 4.1543 - accuracy: 0.6406 - val_loss: 3.5474 - val_accuracy: 0.5775
Epoch 8/3

- 정확도 평가
    - evaluate(): 모델 성능 평가 메서드
        - 첫 번째와 두 번째 매개변수에 입력과 타깃 데이터를 전달
        - compile() 메서드에서 loss 매개변수에 지정한 손실함수의 값과 metrics 매개변수에서 지정한 측정 지표를 출력

In [21]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.4375 - accuracy: 0.8305


[0.4375133514404297, 0.8305084705352783]

- MachineLearning - RandomForest

In [23]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2022)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.8389830508474576